In [ ]:
"""lightningsynth.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/github/torchsynth/torchsynth/blob/lightning-synth/examples/lightningsynth.ipynb

# lightningsynth

Profiling for our synth on GPUs

Make sure you are on GPU runtime

If this hasn't been merged to master yet, run:
```
!pip uninstall -y torchsynth
!pip install git+https://github.com/torchsynth/torchsynth.git@lightning-synth
```
"""

In [ ]:
!pip uninstall -y torchsynth
!pip install git+https://github.com/torchsynth/torchsynth.git@lightning-synth

In [ ]:
!pip install torchvision

In [ ]:
from typing import Any

In [ ]:
import pytorch_lightning as pl
import torch

In [ ]:
# import torchvision.models as models
import torch.autograd.profiler as profiler
import torch.tensor as tensor
from torch import Tensor as T
from tqdm.auto import tqdm

In [ ]:
import torchsynth.module
from torchsynth.config import SynthConfig
from torchsynth.synth import Voice

In [ ]:
gpus = torch.cuda.device_count()
print("Usings %d gpus" % gpus)

In [ ]:
# Note this is the batch size for our synth!
# i.e. this many synth sounds are generated at once
# Not the batch size of the datasets
BATCH_SIZE = 256

In [ ]:
import multiprocessing

In [ ]:
ncores = multiprocessing.cpu_count()
print(f"Using ncores {ncores} for generating batch numbers (low CPU usage)")

In [ ]:
class batch_idx_dataset(torch.utils.data.Dataset):
    def __init__(self, num_batches):
        self.num_batches = num_batches

    def __getitem__(self, idx):
        return idx

    def __len__(self):
        return self.num_batches

In [ ]:
# TODO Add this to torchsynth API
# see https://github.com/torchsynth/torchsynth/issues/154
class TorchSynthCallback(pl.Callback):
    def on_test_batch_end(
        self,
        trainer,
        pl_module: pl.LightningModule,
        outputs: Any,
        batch: Any,
        batch_idx: int,
        dataloader_idx: int,
    ) -> None:
        assert batch.ndim == 1
        _ = pl_module(batch_idx)
        # I don't think the following is correct
        # _ = torch.stack([pl_module(i) for i in batch])

In [ ]:
synth1B = batch_idx_dataset(1024 * 1024 * 1024 // BATCH_SIZE)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(synth1B, num_workers=0, batch_size=1)

In [ ]:
voice = Voice()

In [ ]:
accelerator = None
if gpus == 0:
    use_gpus = None
    precision = 32
else:
    # specifies all available GPUs (if only one GPU is not occupied,
    # auto_select_gpus=True uses one gpu)
    use_gpus = -1
    # Golden cables
    #    # TODO: Change precision?
    #    precision = 16
    if gpus > 1:
        accelerator = "ddp"

In [ ]:
# Use deterministic?
trainer = pl.Trainer(
    precision=precision,
    gpus=use_gpus,
    auto_select_gpus=True,
    accelerator=accelerator,
    deterministic=synthconfig.reproducible,
    max_epochs=0,
    callbacks=[TorchSynthCallback()],
)

In [ ]:
trainer.test(voice, test_dataloaders=test_dataloader)